<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/124_%E3%82%AA%E3%83%96%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E3%81%AE%E7%94%9F%E5%AD%98%E6%9C%9F%E9%96%93%E3%81%A8%E8%A1%A8%E7%8F%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

オブジェクトの生存期間と表現
============================

ガベージコレクション
--------------------

### ガベージコレクションと参照カウント

Python では、新しいオブジェクトが必要になったときにその実体が生成され、ヒープ領域（メモリ上の共有スペース）から必要なメモリが割り当てられる。

オブジェクトがどの変数からも参照されなくなり、不要になった後、そのメモリ領域を解放する必要がある。Python では、この解放作業を処理系が自動的に行う。この仕組みを**ガベージコレクション**（Garbage Collection; GC）と呼ぶ。

オブジェクトが生成され、ガベージコレクションによりメモリ上から消滅するまでの期間を、オブジェクトの**生存期間**(life time)という。

Python（特に CPython）のガベージコレクションは、各オブジェクトへの参照の数を**参照カウント**（reference count）として管理しており、参照カウントが 0 になったオブジェクトは、その時点で即座に破棄されるという方式をとっている。

以下のような局面でオブジェクトが生成されると、参照カウントが 1 になる。

  * 名前への束縛
  * リスト、集合などの要素
  * 辞書などのキーや値

また、これらの局面でオブジェクトが参照されるたびに参照カウントは +1 増え、参照が外れると参照カウントは -1 減る。

既存の変数に別のオブジェクトを代入すると、再束縛が行われるため、新しいオブジェクトの参照カウントは +1 され、同時に、もともとその変数名に束縛されていたオブジェクトの参照カウントは -1 される。変数に `None` を代入した場合も、元のオブジェクトの参照カウントが減る。

del 文はオブジェクトそのものを削除するのではなく、変数名を名前空間から削除することで、そのオブジェクトへの参照を減らすだけである。

`sys.getrefcount(object)` 関数を使うと、`object` に指定したオブジェクトの参照カウントを取得できる。ただし、`sys.getrefcount()` が返す値は一般的に予想より 1 多くなる。これは、`sys.getrefcount()` を呼び出す際に、引数としてオブジェクトが値渡しで一時的に参照され、その分だけ参照カウントが増えるからである。

In [ ]:
%reset -f
import sys
var1 = {1: "spam", 2: "ham", 3: "eggs"}
assert sys.getrefcount(var1) == 2
var2 = var1
assert sys.getrefcount(var1) == 3  # var2 からも参照されたため数が増えた
var2 = None
assert sys.getrefcount(var1) == 2  # var2 が再束縛されたため数が減った
del var1  # 参照カウント 0 となる

なお、Python 3.12 から、特に利用頻度の高い一部の変更不可能オブジェクト（`None`, `True`, `False`, `-5` から `256` までの整数や組み込みデータ型など）は、実行時に参照カウントを更新しないように変更された。対象オブジェクトの参照カウントは、常に 4294967295（=0xFFFFFFFF）固定となる。これにより対象オブジェクトはプログラムの実行中存在し続けるが、並列タスク実行をしている場合にはメモリを共有できるようになったので全体的なメモリ効率は大きく向上する。

In [ ]:
%reset -f
import sys
assert sys.getrefcount(True) == 4294967295
var1 = True
assert sys.getrefcount(True) == 4294967295

### 循環参照と世代別ガベージコレクション

**循環参照**（circular reference）とは、複数のオブジェクトが互いに参照し合っており、参照の輪（サイクル）ができてしまう状態を差す。下図では矢印が参照の向きを表していて赤色のオブジェクトが循環参照になっている（[Wikipedia の記事](https://en.wikipedia.org/wiki/Circular_reference)から引用）。

![](https://upload.wikimedia.org/wikipedia/commons/b/b4/Circular_Reference.svg)

一般に、循環参照が存在すると、どこから処理すればいいかが不明であったり、無限ループになったり、関数の再帰呼び出しで出口がなくなったりする（結果としてスタックオーバーフローが発生する）など、様々な問題を抱えることになる。

Python のメモリ管理の文脈では、循環参照があると参照カウントが 0 にならないという問題がある。

次のコードでは、2 つの変数 `a` と `b` の間で循環参照があり、変数 `a` と `b` を削除してもオブジェクト同士が参照し合っているため参照カウントが 0 にならない。

In [ ]:
%reset -f
import sys
a = []
b = [a]
a.append(b)  # ここで循環参照が発生
print(f"{a=}, {b=}")
assert sys.getrefcount(a) == 3
del b
assert sys.getrefcount(a) == 3  # 変数を削除してもカウントは減らない

a=[[[...]]], b=[[[...]]]


循環参照は参照カウント方式では対応できない。しかし、上図で右下の黒色のオブジェクトがグローバルな名前に束縛されている場合、他のオブジェクトはそのオブジェクトからの参照を辿ってのみ参照可能であるから、もし右下の黒色のオブジェクトの束縛を解除すると、他のオブジェクトは全て参照不可能で不要となるが、参照カウント方式だけではどのオブジェクトも破棄されないことになる。

そこで、CPython は参照カウント方式を基本としつつ、循環参照を検出して回収するための追加 GC を持っている。この GC は、モジュールレベルではグローバルな名前、関数レベルではローカルな名前など、プログラムから何らかの形でアクセス可能なオブジェクトを「到達可能」とみなし、そこから辿れないオブジェクト群を「到達不可能」として回収する。ただし、全ての追跡対象オブジェクトを頻繁にチェックするとパフォーマンスに影響するため、GC はオブジェクトを世代ごとに分類して管理する。これは「ほとんどのオブジェクトはすぐに不要になるが、ある程度長く生存したオブジェクトはその後も長く必要とされる率が高い」という経験則に基づいている。このため、追加 GC は**世代別 GC**（generational garbage collection）と呼ばれる。

世代別 GC では、追跡対象のオブジェクトを「第 0 世代」「第 1 世代」「第 2 世代」の 3 つの世代に分類して管理する。新しく作られたオブジェクトは第 0 世代に置かれ、GC によるスキャンで生き残るたびに上位の世代へ昇格する。GC は若い世代ほど頻繁に、古い世代ほどまれにスキャンすることで、短命なオブジェクトを効率よく回収しつつ、長寿オブジェクトのチェック回数を減らして処理負荷を抑える仕組みになっている。

弱参照（weakref）
-----------------

### 弱参照オブジェクト

参照カウントの働きに気づかないと、不要になったオブジェクトがメモリ上に残り続けることがある。

例えば、次のコードでは、`Player` クラスのインスタンスを `players` リストに登録する形で管理しており、そのリストの中の一部を `card` リストに登録している。`name` 属性が `'武田'` のオブジェクトは両方に登録されるが、次の行で `players` リストから削除されている。これによりオブジェクト自体が削除されるわけではなく、依然として `card` から参照できることがわかる:

In [ ]:
%reset -f
import sys

class Player:
    def __init__(self, id, name):
        self.id = id
        self.name = name

    def __repr__(self):
        return f"Player({self.id}, {self.name})"

players = [Player(32, "武田"), Player(51, "上杉"), Player(66, "織田")]
assert sys.getrefcount(players[0]) == 2
card = [players[0], players[1]]
assert sys.getrefcount(players[0]) == 3
del players[0]
# players からは "武田" が除去されるが、card では依然として "武田" を参照できている
print(f"{players = }")
print(f"{card = }")

players = [Player(51, 上杉), Player(66, 織田)]
card = [Player(32, 武田), Player(51, 上杉)]


このように、オブジェクトが複数の変数や他のオブジェクトによって参照されている場合、それらの参照をすべて解除して参照カウントが 0 になるようにしないと、メモリ上にはオブジェクトが存在し続ける。

この問題を解決するため、標準ライブラリの `weakref` モジュールは、参照カウントを増やさない参照をサポートする。このような参照を**弱参照**（weak reference）と呼ぶ。弱参照と区別するため、普通の参照（参照カウントが増える参照）を**強参照**と呼ぶことがある。

ただし、全てのオブジェクトが弱参照で参照できるわけではない。以下のオブジェクトは弱参照をサポートする。

  * クラスインスタンス（`__slots__` 属性を持つクラスを除く）
  * （C ではなく） Python で書かれた関数
  * インスタンスメソッド
  * `set` オブジェクト
  * `frozenset` オブジェクト
  * ファイルオブジェクト
  * ジェネレーターオブジェクト
  * `socket` オブジェクト（`socket` モジュール）
  * `array` オジェクト（`array` モジュール）
  * `deque` オブジェクト（`deque` モジュール）
  * 正規表現パターンオブジェクト（`re` モジュール）

`__slots__` をクラスで定義した場合は、`__slots__ = ('x', 'y', '__weakref__')` のように 明示的に `'__weakref__'` を含めたときに限って、弱参照をサポートする。一般のクラスインスタンスでは、この `__weakref__` 属性が自動的に追加される。

いくつかの組み込み型 `int`, `float`, `str`, `bytes`, `bytearray`, `list`, `tuple`, `dict` は弱参照を直接サポートしないことに注意する。`list` と `dict` に限っては、サブクラス化を行えば、そのサブクラスが弱参照をサポートする。

`weakref.ref()` は弱参照を表現する、呼び出し可能型のクラスであり、コンストラクタは次のとおり。

``` python
weakref.ref(object[, callback])
```

第 1 引数は、弱参照をサポートするオブジェクトでなければならない。このオブジェクトは、まだ生きているなら参照オブジェクトの呼び出しで取り出せる。オブジェクトがもはや生きていないならば、参照オブジェクトを呼び出したときに `None` を返す。

同じオブジェクトに対してたくさんの弱参照オブジェクトを作成することができ、参照カウントは増えない。そのオブジェクトは、唯一の強参照が解除されて参照カウントが 0 になると即座に破棄される（循環参照がある場合は世代別 GC が回収する）。

既にガベージコレクションされたオブジェクトへの弱参照を呼び出すと、弱参照は無効（dead）な状態となっており、`None` が返される。

省略可能な第 2 引数にコールバック関数を指定すると、弱参照が無効になった直後にそのコールバックが呼び出される。このとき、弱参照オブジェクト自身がコールバック関数の唯一の引数として渡される。

In [ ]:
%reset -f
import weakref
import sys

class Player:
    def __init__(self, id, name):
        self.id = id
        self.name = name

    def __repr__(self):
        return f"Player({self.id}, {self.name})"

players = [Player(32, "武田"), Player(51, "上杉"), Player(66, "織田")]
assert sys.getrefcount(players[0]) == 2
card = [weakref.ref(players[0]), weakref.ref(players[1])]  # 弱参照を登録
assert sys.getrefcount(players[0]) == 2  # 参照カウントが増えない
# 呼び出し形式で弱参照から "武田" にアクセス可能
assert card[0]().name == "武田"
del players[0]
# players から "武田" が除去されると、弱参照からはアクセスできない
assert card[0]() is None

次のコードは、辞書が弱参照をサポートしないので、サブクラス化した上でそのインスタンスから弱参照を作成している。

In [ ]:
%reset -f
import weakref
import sys

# dict のサブクラス化（弱参照のサポート）
class Dict(dict):
    pass

x = Dict(name="hoge")
assert sys.getrefcount(x) == 2
x_ref = weakref.ref(x)  # 弱参照
assert sys.getrefcount(x) == 2  # 参照カウントは増えない
assert x_ref()["name"] == "hoge"  # x_ref() でインスタンスを参照
del x
assert x_ref() is None  # del x のあとはインスタンスを参照できない

### プロキシ

弱参照オブジェクト（`weakref.ref`）を使って元のオブジェクトにアクセスする場合は、呼び出して取得する必要がある。そのため、次の関数で作成されるプロキシを使うほうが便利である。

``` python
weakref.proxy(object[, callback])
```

プロキシは内部的に弱参照を保持する薄いラッパーであり、自身が第 1 引数のオブジェクトのように振る舞うため、`proxy.attr` や `proxy.method()` のように直接アクセスできる。

元のオブジェクトがガベージコレクションされた後にプロキシへアクセスすると `ReferenceError` が発生する。

In [ ]:
%reset -f
import weakref
import sys

class Player:
    def __init__(self, id, name):
        self.id = id
        self.name = name

    def __repr__(self):
        return f"Player({self.id}, {self.name})"

players = [Player(32, "武田"), Player(51, "上杉"), Player(66, "織田")]
assert sys.getrefcount(players[0]) == 2
card = [weakref.proxy(players[0]), weakref.proxy(players[1])]  # 弱参照を登録
assert sys.getrefcount(players[0]) == 2  # 参照カウントが増えない
# プロキシは呼び出し形式によらずに "武田" にアクセス可能
assert card[0].name == "武田"
del players[0]
try:
    print(card[0])
except Exception as err:
    print(f"{type(err).__name__}: {err}")

ReferenceError: weakly-referenced object no longer exists


### WeakKeyDictionary・WeakValueDictionary・WeakSet

`weakref` モジュールは、弱参照を扱うデータ型も提供している。

``` python
weakref.WeakKeyDictionary([dict])
```

キーを弱参照するマッピングクラス。キーへの強参照がなくなったときに、辞書のエントリは捨てられる。

``` python
weakref.WeakValueDictionary([dict])
```

値を弱参照するマッピングクラス。値への強参照が存在しなくなったときに、辞書のエントリは捨てられる。

`weakref.WeakKeyDictionary` 型と `weakref.WeakValueDictionary` 型の型アノテーションは、`dict` と同様に、添字表記（[]）を使ってキーの型と値の型を指定しなければならない。

``` python
weakref.WeakSet([elements])
```

要素への弱参照を持つ集合型。要素への強参照が無くなったときに、その要素は削除される。`weakref.WeakSet` 型の型アノテーションは、`set` と同様。

弱参照の一般的なユースケースのひとつに、木構造がある。木構造では、「親→子」と「子→親」のように双方向参照（強参照）を作ると循環参照が発生するため、節点（ノード）を削除しても回収が遅れるという問題がある。どちらか一方向を弱参照とすることで循環を断ち切れる。

次のコードでは、`Node` クラスを使って木構造を実装している。`weakref.WeakValueDictionary` を使って、親ノードが子ノードへの弱参照を辞書として保持している。子ノードが不要になればすぐに GC される。

In [ ]:
%reset -f
from weakref import WeakValueDictionary
from typing import Self

class Node:
    def __init__(self, value: str):
        self.value = value
        self._parent = None
        self._children = WeakValueDictionary()  # type: WeakValueDictionary[str, Self]

    def __repr__(self):
        return "Node({!r:})".format(self.value)

    @property
    def parent(self):
        return self._parent

    @property
    def children(self):
        return list(self._children.items())

    def add_child(self, key: str, child: Self):
        self._children[key] = child
        child._parent = self

root = Node("parent")
n1 = Node("child one")
n2 = Node("child two")
root.add_child("n1", n1)
root.add_child("n2", n2)
assert n1.parent == root and n2.parent == root
print(root.children)  # [('n1', Node('child one')), ('n2', Node('child two'))]

del n1
print(root.children)  # [('n2', Node('child two'))]

[('n1', Node('child one')), ('n2', Node('child two'))]
[('n2', Node('child two'))]


### ファイナライザ

**ファイナライザ**（finalizer）は、ガベージコレクションの機能を持つ言語において不要オブジェクトが回収される前に自動的に呼び出されるメソッドを指す。似たものに**デストラクタ**（destructor）があるが、これはオブジェクトを「直ちに」削除する際に自動的に呼び出されるメソッドのことである。

Python では、オブジェクトが `__del__()` メソッドを持っている場合、これをファイナライザとして扱う。つまり、オブジェクトの参照カウントが 0 まで落ちたときに `__del__()` メソッドがあれば呼び出される。del 文は参照カウントを減らすだけで、直接 `__del__()` メソッドを呼び出すわけではないことに注意する。

In [ ]:
%reset -f
class A:
    def __del__(self):
        print("後処理")

a = A()
del a  # 参照カウントが 0 になった

後処理


CPython の実装では `__del__()` メソッドは 1 回しか呼び出されない。例えば `__del__()` メソッドが破棄しようとしているインスタンスへの新しい参照を作っても、CPython では 1 回だけしか破棄を遅らせることができない。

`__del__()` メソッドは、次のような問題がある。

  * インタプリタ終了時にまだ存在しているオブジェクトについては、`__del__()` メソッドが呼び出されることは保証されていない。
  * 循環参照が存在する場合、`__del__()` メソッドが呼び出されるタイミングは予測不能となる。オブジェクトがインタプリタ終了時まで存続して、`__del__()` メソッドが一度も呼ばれずにプログラムが終了することがある。

`weakref` モジュールは、より信頼性のある `weakref.finalize` クラスを提供している。コンストラクタは次のとおり。

``` python
weakref.finalize(obj, func, /, *args, **kwargs)
```

生成される `weakref.finalize` オブジェクトは、呼び出し可能で、第 1 引数のオブジェクトのファイナライザとして振る舞う。通常の弱参照とは異なり、このファイナライザは呼び出されるまで「生きている」と見なされ、呼び出された後には無効（dead）な状態となる。生きているファイナライザを呼び出すと、`func(*arg, **kwargs)` を評価した結果を返す。一方、無効なファイナライザを呼び出すと `None` を返す。

`weakref.finalize` オブジェクトは、参照しているオブジェクトが残ったままプログラムが終了する場合にも呼び出される。

`weakref.finalize` オブジェクトは読み取り専用の `alive` プロパティを持ち、これは自身が生きている場合には `True`、そうでない場合には `False` を返す。このプロパティを確認することで、参照しているオブジェクトが「まだ生存しているかどうか」を確認できる。

In [ ]:
%reset -f
import weakref

class Sample:
    pass

def finalize_sample(*args, **kwargs):
    print("後処理")

sample = Sample()
sample_list = [sample]
finalize = weakref.finalize(sample, finalize_sample)
del sample
assert finalize.alive
sample_list.clear()
assert not finalize.alive

後処理


データの整形表示
----------------

### pprint ###

標準ライブラリの `pprint` モジュールが提供する `pprint.pprint()` 関数は、何重にもネストされた辞書や、要素数が長いリストのように、人間が読みにくい構造のデータを整形して出力する。

``` python
pprint.pprint(object, stream=None, indent=1, width=80, depth=None, *, compact=False, sort_dicts=True, underscore_numbers=False)
```

| 引数 | 意味 |
|:--|:--|
| `object` | 出力するデータを指定する |
| `stream` | 出力先のファイルオブジェクトを指定する。`None`（デフォルト値）ならば、標準出力 `sys.stdout` に出力される |
| `indent` | ネストされたオブジェクトの子要素を出力するときのインデント数を指定する |
| `width` | 出力幅を指定する |
| `depth` | ネストされたオブジェクトを出力する際の、最大レベル数を指定する。`None`（デフォルト値）ならば、すべてのレベルを出力する |
| `compact` | キーワード専用引数。`True` の場合、各行に `width` の幅に収まるだけの要素が出力される。`False` の場合、1 行ごとに 1 要素が出力される |
| `sort_dicts` | キーワード専用引数。`True` の場合、キーがソートされた状態で出力される。`False` の場合、挿入順に出力される |
| `underscore_numbers` | キーワード専用引数。`True` の場合、整数は千の位の区切り文字としてアンダースコア `_` が使用される。`False` の場合、アンダースコアは使<br />用されない |

この関数でネストされた辞書を整形して出力する場合、デフォルトではキーがソートされた状態で出力されることに注意する。挿入順に出力するには、`sort_dicts=False` を指定する必要がある。

In [ ]:
# ネストした辞書
import pprint
users = {
    'user1': {
        'name': '山田 太郎',
        'age': 30,
        'city': '東京'
    },
    'user2': {
        'name': '鈴木 花子',
        'age': 25,
        'city': '大阪'
    },
    'user3': {
        'name': '田中 健太',
        'age': 45,
        'city': '福岡'
    }
}
pprint.pprint(users, width=30)
print('>>sort_dicts=False')
pprint.pprint(users, width=30, sort_dicts=False)

{'user1': {'age': 30,
           'city': '東京',
           'name': '山田 太郎'},
 'user2': {'age': 25,
           'city': '大阪',
           'name': '鈴木 花子'},
 'user3': {'age': 45,
           'city': '福岡',
           'name': '田中 健太'}}
>>sort_dicts=False
{'user1': {'name': '山田 太郎',
           'age': 30,
           'city': '東京'},
 'user2': {'name': '鈴木 花子',
           'age': 25,
           'city': '大阪'},
 'user3': {'name': '田中 健太',
           'age': 45,
           'city': '福岡'}}


In [ ]:
# 長いリスト
import pprint
stuff = ["spam", "eggs", "lumberjack", "knights", "ni"]
stuff.insert(0, stuff)
pprint.pprint(stuff)

[<Recursion on list with id=134865144186240>,
 'spam',
 'eggs',
 'lumberjack',
 'knights',
 'ni']


``` python
pprint.pformat(object, indent=1, width=80, depth=None, *, compact=False, sort_dicts=True, underscore_numbers=False)
```

この関数は、`pprint.pprint()` 関数と同じくオブジェクトを整形するが、標準出力に印字するのではなく、戻り値として文字列を返す。

### tabulate ###

サードパーティ製の [tabulate](https://pypi.org/project/tabulate/) は、ネストされたリストなど一定のデータ構造を表形式に整形して出力する関数を提供する。ライセンスは MIT License。インストールは次のとおり。

``` shell
pip install tabulate
```

``` python
tabulate(tabular_data, headers=(), tablefmt="simple", floatfmt=_DEFAULT_FLOATFMT, intfmt=_DEFAULT_INTFMT, numalign=_DEFAULT_ALIGN,
         stralign=_DEFAULT_ALIGN, missingval=_DEFAULT_MISSINGVAL, showindex="default", disable_numparse=False, colglobalalign=None,
         colalign=None, preserve_whitespace=False, maxcolwidths=None, headersglobalalign=None, headersalign=None, rowalign=None,
         maxheadercolwidths=None, break_long_words=_BREAK_LONG_WORDS, break_on_hyphens=_BREAK_ON_HYPHENS)
```

この関数は、一定のデータ構造を表形式に整形したテキストを生成して返す。以下のデータ構造をサポートする。

| データ構造 | 例 |
|:---|:---|
| ネストされたリスト | `[['Name', 'Age'], ['Alice', 24], ['Bob', 19]]` |
| リストを含む辞書 | `{'Name': ['Alice', 'Bob'], 'Age': [24, 19]}` |
| 辞書のリスト | `[{'Name': 'Alice', 'Age': 24}, {'Name': 'Bob', 'Age': 19}]` |
| dataclass のリスト | `[Person('Alice', 24), Person('Bob', 19)]` |

関数の主な引数は、以下の通り。

| 引数 | 意味 |
|:---|:---|
| `tabular_data` | 入力データ |
| `headers` | 表のヘッダーを指定する<br /><br />・空リストや空タプル: ヘッダーを表示しない（デフォルト）<br /><br />・文字列のリストまたはタプル: ヘッダーとして表示したい文字列を順序通りにリストまたはタプルで指定する<br />　・`tabular_data` がネストされたリストの場合、各リストの順序通りに要素がヘッダーに対応付けられる<br />　・`tabular_data` がリストを含む辞書や、辞書のリストである場合、ヘッダーと一致するキーの値がヘッダーに対応付けられる<br />　・`tabular_data` が dataclass のリストである場合、ヘッダーと一致するフィールドの値がヘッダーに対応付けられる<br /><br />・`'firstrow'`: `tabular_data` がネストされたリストの場合、最初の内側のリストを表のヘッダーとして扱う<br /><br />・`'keys'`: `tabular_data` がリストを含む辞書や、辞書のリストである場合、各キーを表のヘッダーとする |
| `tablefmt` | 表の書式を文字列で指定する。デフォルトは `'simple'` |
| `floatfmt` | `float` 型データに対する書式指定。デフォルトは `'g'` |
| `intfmt` | `int` 型データに対する書式指定。デフォルトは 10 進数 |
| `numalign` | 数値の列の配置。`'right'`（右揃え）, `'center'`（中央揃え）, `'left'`（左揃え）, `'decimal'` |
| `stralign` | 文字列の列の配置。`'right'`（右揃え）, `'center'`（中央揃え）, `'left'`（左揃え） |
| `showindex` | `True` の場合、自動連番でインデックスを表示する。イテラブルの場合、表示する任意のインデックスを指定する |
| `colalign` | シーケンスで列ごとの配置を指定する。`'right'`（右揃え）, `'center'`（中央揃え）, `'left'`（左揃え） |

書式 `tablefmt='simple'` の例:

In [ ]:
# ネストされたリストを整形出力
from tabulate import tabulate
table = [["Sun",696000,1989100000], ["Earth",6371,5973.6], ["Moon",1737,73.5], ["Mars",3390,641.85]]
print(tabulate(table, headers=["Planet","R (km)", "mass (x 10^29 kg)"]))

Planet      R (km)    mass (x 10^29 kg)
--------  --------  -------------------
Sun         696000           1.9891e+09
Earth         6371        5973.6
Moon          1737          73.5
Mars          3390         641.85


書式 `tablefmt='plain'` の例:

In [ ]:
# ネストされたリストを整形出力
from tabulate import tabulate
print(tabulate([["Name", "Age"], ["Alice", 24], ["Bob", 19]],
               headers='firstrow', tablefmt='plain'))

Name      Age
Alice      24
Bob        19


書式 `tablefmt='grid'` の例:

In [ ]:
# リストを含む辞書を整形出力
from tabulate import tabulate
print(tabulate({"Name": ["Alice", "Bob"], "Age": [24, 19]},
               headers="keys", tablefmt='grid'))

+--------+-------+
| Name   |   Age |
+========+=======+
| Alice  |    24 |
+--------+-------+
| Bob    |    19 |
+--------+-------+


書式 `tablefmt='psql'` の例:

In [ ]:
# 辞書のリストを整形出力
from tabulate import tabulate
print(tabulate([{"Name": "Alice", "Age": 24}, {"Name": "Bob", "Age": 19}],
               headers="keys", tablefmt='psql'))

+--------+-------+
| Name   |   Age |
|--------+-------|
| Alice  |    24 |
| Bob    |    19 |
+--------+-------+


書式 `tablefmt='pretty'` の例:

In [ ]:
# dataclass のリストを整形出力
from dataclasses import dataclass, fields
from tabulate import tabulate

@dataclass
class Person:
    name: str
    age: bool

print(tabulate([Person("Alice", 24), Person("Bob", 19)],
               headers=[f.name for f in fields(Person)],
               tablefmt='pretty'))

+-------+-----+
| name  | age |
+-------+-----+
| Alice | 24  |
|  Bob  | 19  |
+-------+-----+


書式 `tablefmt='github'` の例:

In [ ]:
# インデックスを表示
from tabulate import tabulate
print(tabulate({"Name": ["Alice", "Bob"], "Age": [24, 19]},
               headers="keys", tablefmt='github', showindex=True))

|    | Name   |   Age |
|----|--------|-------|
|  0 | Alice  |    24 |
|  1 | Bob    |    19 |


書式 `tablefmt='pipe'` の例:

In [ ]:
# 列ごとの配置を指定
from tabulate import tabulate
print(tabulate({"Name": ["Alice", "Bob"], "Age": [24, 19], "Gender": ["female", "male"]},
               headers="keys", tablefmt='pipe', colalign=["left", "right", "center"]))

| Name   |   Age |  Gender  |
|:-------|------:|:--------:|
| Alice  |    24 |  female  |
| Bob    |    19 |   male   |


書式 `tablefmt='html'` の例:

In [ ]:
# float 型データに対する書式指定
import math
from tabulate import tabulate
from IPython.display import HTML
display(HTML(tabulate({"constant": ["e", "pi"], "value": [math.e, math.pi]},
             headers="keys", tablefmt='html', floatfmt='.2f')))

constant,value
e,2.72
pi,3.14


直列化（シリアライズ）
----------------------

プログラム内で扱う複雑なデータ構造やオブジェクトを、一列のバイト列や文字列などの形式に変換することを**直列化**または**シリアライズ**（serialize）といい、復元することを**非直列化**または**デシリアライズ**（deserialize）という。

シリアライズしたものをファイルに保存すると、オブジェクトのデータ構造とデータの値をそのまま保存することができる。ファイルを読み込み、デシリアライズすると、元のオブジェクトを復元できる。このようにオブジェクトを半永久的に保存し、いつでも復元できるようにすることを**永続化**（perpetuation）という。

### pickle プロトコル ###

**pickle プロトコル**は、Python のオブジェクトをバイト列で直列化するための共通ルールである。pickle プロトコルを使用して直列化することを **pickle 化**、非直列化することを**非 pickle 化**と呼ぶ。ファイル名の拡張子は `.pickle` とすることが多い。

pickle は極めて多くの Python オブジェクトを表現できる。以下の型が pickle 化可能とされる。

  * 組み込み定数（`None`, `True`, `False`, `Ellipsis`, `NotImplemented`）
  * 整数、浮動小数点数、複素数
  * 文字列、バイト列、バイト配列
  * pickle 化可能なオブジェクトからなるタプル、リスト、集合および辞書
  * モジュールのトップレベルで定義された関数（`def` で定義されたもののみで `lambda` で定義されたものは含まない）
  * モジュールのトップレベルで定義されているクラス
  * `__getstate__()` メソッドを呼び出した結果が pickle 化可能であるようなクラスのインスタンス

pickle は Python 固有のバイナリ形式であり、相互運用可能なフォーマットではない。

現在 pickle プロトコルにはバージョン 0 から 5 までの 6 種類がある。より高いバージョンのプロトコルを使用するほど、保存、送受信に最適化されたバイナリ形式となるが、作成された pickle を読み込むためにより高い Python のバージョンが必要になる。

標準ライブラリの `pickle` モジュールは、全てのバージョンの pickle プロトコルを実装している。よく利用されるバージョンにはモジュール定数（整数）を用意している。

| モジュール定数 | 意味 | 必要な Python バージョン |
|:---|:---|:---|
| `pickle.HIGHEST_PROTOCOL` | pickle プロトコルのバージョン 5 | Python 3.8 以上 |
| `pickle.DEFAULT_PROTOCOL` | pickle プロトコルのバージョン 4 | Python 3.4 以上 |

### pickle 化 ###

``` python
pickle.dump(obj, file, protocol=None, *, fix_imports=True, buffer_callback=None)
```

この関数は、オブジェクト `obj` を pickle 化し、すでに開いているファイルオブジェクト `file` に書き込む。オプションの第 3 引数 `protocol` に、使用する pickle プロトコルのバージョン（整数）を指定できる。指定されない場合、`pickle.DEFAULT_PROTOCOL` が使用される。

`file` を与える `open()` 関数はバイナリモードを指定する必要がある。`pickle.dump()` 関数の基本的な使い方は次のようになる。

``` python
import pickle
with open(filepath, 'wb') as f:
    pickle.dump(obj, f)
```

``` python
pickle.dumps(obj, protocol=None, *, fix_imports=True, buffer_callback=None)
```

この関数は、オブジェクト `obj` を pickle 化したバイト列を返す。オプションの第 2 引数 `protocol` の意味は `pickle.dump()` 関数の同名の引数と同じ。

ファイルオブジェクトやジェネレーターは pickle 化できないので、これらを pickle 化しようとすると `pickle.PicklingError` 例外が発生する。

### 非 pickle 化 ###

非 pickle 化により任意のコードを実行することが可能である。**信頼できない提供元からのデータや、改竄された可能性のあるデータの非 pickle 化は絶対に行ってはいけない**。

``` python
pickle.load(file, *, fix_imports=True, encoding='ASCII', errors='strict', buffers=None)
```

この関数は、すでに開いている pickle ファイルのファイルオブジェクト `file` を読み込み、非 pickle 化した結果のオブジェクトを返す。pickle プロトコルのバージョンは自動的に検出される。

`file` を与える `open()` 関数はバイナリモードを指定する必要がある。`pickle.dump()` 関数の基本的な使い方は次のようになる。

``` python
import pickle
with open(filepath, 'rb') as f:
    x = pickle.load(f)
```

``` python
pickle.loads(data, /, *, fix_imports=True, encoding='ASCII', errors='strict', buffers=None)
```

この関数は、pickle 化表現であるバイト列 `data` を非 pickle 化した結果のオブジェクトを返す。pickle プロトコルのバージョンは自動的に検出される。

In [ ]:
import pickle
import datetime as dt

data = {
    "a": [1, 2.0, 3 + 4j],
    "b": ("character string", b"byte string"),
    "c": {None, True, False},
    "d": dt.datetime(2023, 10, 31, 21, 30, tzinfo=dt.timezone(dt.timedelta(hours=9), "JST")),
}

# シリアライズ
serialized = pickle.dumps(data)
print(f"{serialized=}", "\n")

# datetime モジュールをアンインポートする
del dt

# デシリアライズ
data2 = pickle.loads(serialized)

# datetime モジュールを import していなくてもエラーにならない
print(f"{data2['d']:%Y年%m月%d日 %H時%M分} ({data2['d'].tzname()})")
data2

serialized=b'\x80\x04\x95\xd8\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x01a\x94]\x94(K\x01G@\x00\x00\x00\x00\x00\x00\x00\x8c\x08builtins\x94\x8c\x07complex\x94\x93\x94G@\x08\x00\x00\x00\x00\x00\x00G@\x10\x00\x00\x00\x00\x00\x00\x86\x94R\x94e\x8c\x01b\x94\x8c\x10character string\x94C\x0bbyte string\x94\x86\x94\x8c\x01c\x94\x8f\x94(N\x88\x89\x90\x8c\x01d\x94\x8c\x08datetime\x94\x8c\x08datetime\x94\x93\x94C\n\x07\xe7\n\x1f\x15\x1e\x00\x00\x00\x00\x94h\x0f\x8c\x08timezone\x94\x93\x94h\x0f\x8c\ttimedelta\x94\x93\x94K\x00M\x90~K\x00\x87\x94R\x94\x8c\x03JST\x94\x86\x94R\x94\x86\x94R\x94u.' 

2023年10月31日 21時30分 (JST)


{'a': [1, 2.0, (3+4j)],
 'b': ('character string', b'byte string'),
 'c': {False, None, True},
 'd': datetime.datetime(2023, 10, 31, 21, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), 'JST'))}

このコードからわかるように、`datetime`, `date`, `time` の各インスタンスは特別扱いで、非直列化のときに `import datetime` しなくても復元できる。しかし、他のモジュールのトップレベルで定義された関数やクラスはそうではない。それらも pickle 化できるが、名前（完全修飾名）のみが pickle 化され、そのコードや属性は pickle 化されない。したがって、非 pickle 化を行うモジュールにおいて、pickle 化した関数やクラスを定義するモジュールが import されていないと、例外が発生する。

In [ ]:
import pickle

class MyClass:
    def method(self):
        return 'This is MyClass.'

m1 = MyClass()
m1.a = 100
b = pickle.dumps(m1)

m2 = pickle.loads(b)
assert m2.method() == 'This is MyClass.'
assert m2.a == 100

del MyClass
try:
    m3 = pickle.loads(b)
except Exception as err:
    print(f"{type(err).__name__}: {err}")

AttributeError: Can't get attribute 'MyClass' on <module '__main__'>


### クラスインスタンスの pickle 化 ###

pickle はオブジェクトを直列化するとき、次の手順で状態を取得する。

  1. **`__getstate__()` メソッドが定義されていれば呼び出す**  
このメソッドが返したオブジェクト（通常は辞書など）が「直列化される内容」になる
  2. **なければ `__dict__` を使う**  
通常は `obj.__dict__` に入っている属性がそのまま直列化される
  3. **`__slots__` が定義されているクラスの場合**  
`__slots__` の値も直列化対象にする

非直列化するときは `__setstate__()` が使える。

  * `__setstate__(state)` を実装しておくと、pickle は復元時に state（`__getstate__()` の戻り値）を渡す。
  * もし `__setstate__()` がなければ、`__dict__` にそのまま反映する

In [ ]:
import pickle

class MyClass:
    def __init__(self):
        self.data = [1, 2, 3]
        self.secret = "hidden"

    def __getstate__(self):
        # secret属性は保存しない
        state = self.__dict__.copy()
        del state['secret']
        return state

    def __setstate__(self, state):
        # 復元時にsecretをデフォルト値に設定
        self.__dict__.update(state)
        self.secret = "default"

obj = MyClass()
data = pickle.dumps(obj)
new_obj = pickle.loads(data)

assert new_obj.data == [1, 2, 3]
assert new_obj.secret == "default"